In [1]:
import aug_util as aug
import wv_util as wv
import matplotlib.pyplot as plt
import numpy as np
import csv
from PIL import Image
import os
%matplotlib inline

In [2]:
path_to_data = '../../../xview_data'

### Get all bounding boxes

In [3]:
# Get all coords, chips, and classes
coords, chips, classes = wv.get_labels('{}/xView_train.geojson'.format(path_to_data))

100%|██████████| 601937/601937 [00:03<00:00, 180363.29it/s]


In [4]:
# Sanity check stats
print('Shapes')
print('coords: {}'.format(coords.shape))
print('chips: {}'.format(chips.shape))
print('classes: {}'.format(classes.shape))

print('\nAn example')
print('coords[1]: {}'.format(coords[1]))
print('chips[1]: {}'.format(chips[1]))
print('classes[1]: {}'.format(classes[1]))

print('\nNum unique chips: {}'.format(len(set(chips))))
print('Num unique classes: {}'.format(len(set(classes))))

Shapes
coords: (601937, 4)
chips: (601937,)
classes: (601937,)

An example
coords[1]: [2720. 2233. 2760. 2288.]
chips[1]: 2355.tif
classes[1]: 73.0

Num unique chips: 847
Num unique classes: 62


### Create names (classes) file
Step 3 of "Train Custom Data" [wiki](https://github.com/ultralytics/yolov3/wiki/Example:-Transfer-Learning). Remap class numbers to be consecutive and to start from 0, then write all class labels to `*.names` file.

In [5]:
old_to_new_labels = {}
old_labels = {}
new_labels = {}
new_num = 0

# Remap all explicitly named classes
with open('xview_class_labels.txt') as f:
    for row in csv.reader(f):
        arr = row[0].split(":")
        old_num = int(arr[0])
        label_str = arr[1]
        
        old_to_new_labels[old_num] = new_num 
        old_labels[old_num] = label_str
        new_labels[new_num] = label_str
        
        new_num += 1

# Remap unnamed classes
for c in set(classes):
    if c not in old_labels:
        old_to_new_labels[int(c)] = new_num
        new_labels[new_num] = "Unknown label {}".format(new_num)
        new_num += 1

In [6]:
print(old_to_new_labels)

{11: 0, 12: 1, 13: 2, 15: 3, 17: 4, 18: 5, 19: 6, 20: 7, 21: 8, 23: 9, 24: 10, 25: 11, 26: 12, 27: 13, 28: 14, 29: 15, 32: 16, 33: 17, 34: 18, 35: 19, 36: 20, 37: 21, 38: 22, 40: 23, 41: 24, 42: 25, 44: 26, 45: 27, 47: 28, 49: 29, 50: 30, 51: 31, 52: 32, 53: 33, 54: 34, 55: 35, 56: 36, 57: 37, 59: 38, 60: 39, 61: 40, 62: 41, 63: 42, 64: 43, 65: 44, 66: 45, 71: 46, 72: 47, 73: 48, 74: 49, 76: 50, 77: 51, 79: 52, 83: 53, 84: 54, 86: 55, 89: 56, 91: 57, 93: 58, 94: 59, 75: 60, 82: 61}


In [7]:
# Create new *.names file listing all of the names for the classes in our dataset
label_file = open("{}/yolo_data/xview.names".format(path_to_data), "a")
for i in new_labels:
    label_file.write('{}\n'.format(new_labels[i]))
label_file.close()

In [8]:
# Remap classes array
remapped_classes = list(map(lambda c: old_to_new_labels[int(c)], classes))
remapped_classes = np.array(remapped_classes)

In [9]:
print(remapped_classes.shape)

(601937,)


### Chip each image
Step 1 of "Train Custom Data" [wiki](https://github.com/ultralytics/yolov3/wiki/Example:-Transfer-Learning). Output all chip tifs and associated label txt files. Each image's label file must be locatable by simply replacing `/images/*.jpg` with `/labels/*.txt` in its pathname. Additional requirements:
* One file per image (if no objects in image, no label file is required).
* One row per object.
* Each row is `class x_center y_center width height` format.
* Box coordinates must be in normalized xywh format (from 0 - 1).
* Classes are zero-indexed (start from 0).

In [14]:
chip_size = (500, 500)
path_to_save = '{}/yolo_data'.format(path_to_data)

a = 0
# For each image in directory
for img_name in os.listdir('{}/train_images'.format(path_to_data)):
    if not img_name.endswith(".tif") or img_name.startswith("."):
        continue
    
    # Get full image
    print('{}: {}'.format(a, img_name))
    full_path = '{}/train_images/{}'.format(path_to_data, img_name)
    arr = wv.get_image(full_path)
    
    # Get chip arrays, bounding boxes, and classes
    c_img, c_box, c_cls = wv.chip_image(img=arr, coords=coords[chips==img_name], classes=classes[chips==img_name], shape=chip_size)
    num_chips = c_img.shape[0]
#     print("\tNum chips: {}".format(num_chips))
        
    for i in range(num_chips):
        filename = '{}_{}'.format(img_name.split('.')[0], i)
        
        # Write chip JPG
        chip_arr = c_img[i]
        chip_img = Image.fromarray(chip_arr)
        resized = chip_img.resize((416, 416))
        resized.save("{}/images/{}.jpg".format(path_to_save, filename), "JPEG")
        
        # Write chip label txt
        chip_bboxes = c_box[i]
        chip_classes = c_cls[i]
        num_bboxes = len(chip_classes)
#         print('\t\tNum bounding boxes: {}'.format(num_bboxes))
        
        labels_str = ""
        for j in range(num_bboxes):
            # bounding box format: xmin, ymin, xmax, ymax
            xmin, ymin, xmax, ymax = chip_bboxes[j]
            bbox_class = int(chip_classes[j])
            
            # row format: class x_center y_center width height
            if bbox_class == 0:
                labels_str += "0 0 0 0 0\n"
            else:
                labels_str += '{} {} {} {} {}\n'.format(old_to_new_labels[bbox_class],
                                                       (xmax + xmin)/2 / chip_size[1], 
                                                       (ymax + ymin)/2 / chip_size[0],
                                                       (xmax - xmin) / chip_size[1],
                                                       (ymax - ymin) / chip_size[0])
        
        label_file = open("{}/labels/{}.txt".format(path_to_save, filename), "a")
        label_file.write(labels_str)
        label_file.close()
        
    a += 1

0: 303.tif
1: 136.tif
2: 302.tif
3: 73.tif
4: 1896.tif
5: 1590.tif
6: 1579.tif
7: 469.tif
8: 181.tif
9: 556.tif
10: 1463.tif
11: 498.tif
12: 1351.tif
13: 87.tif
14: 612.tif
15: 125.tif
16: 1906.tif
17: 727.tif
18: 1165.tif
19: 1208.tif
20: 1280.tif
21: 546.tif
22: 2168.tif
23: 394.tif
24: 1046.tif
25: 1067.tif
26: 315.tif
27: 1581.tif
28: 2207.tif
29: 2162.tif
30: 684.tif
31: 2179.tif
32: 1976.tif
33: 815.tif
34: 1442.tif
35: 1827.tif
36: 2055.tif
37: 574.tif
38: 1357.tif
39: 375.tif
40: 1329.tif
41: 1900.tif
42: 1921.tif
43: 2497.tif
44: 658.tif
45: 1447.tif
46: 2110.tif
47: 1606.tif
48: 20.tif
49: 95.tif
50: 2214.tif
51: 604.tif
52: 1888.tif
53: 1092.tif
54: 492.tif
55: 650.tif
56: 1908.tif
57: 1769.tif
58: 767.tif
59: 1585.tif
60: 2009.tif
61: 1418.tif
62: 744.tif
63: 895.tif
64: 1466.tif
65: 1694.tif
66: 1090.tif
67: 2514.tif
68: 2436.tif
69: 10.tif
70: 2237.tif
71: 805.tif
72: 532.tif
73: 462.tif
74: 1036.tif
75: 2361.tif
76: 2112.tif
77: 84.tif
78: 1379.tif
79: 291.tif
80: 2115.t

611: 1385.tif
612: 2314.tif
613: 1362.tif
614: 870.tif
615: 1703.tif
616: 1397.tif
617: 415.tif
618: 2122.tif
619: 43.tif
620: 1912.tif
621: 540.tif
622: 764.tif
623: 46.tif
624: 1568.tif
625: 765.tif
626: 1468.tif
627: 1823.tif
628: 2542.tif
629: 287.tif
630: 1309.tif
631: 720.tif
632: 2021.tif
633: 2303.tif
634: 86.tif
635: 1185.tif
636: 674.tif
637: 887.tif
638: 520.tif
639: 481.tif
640: 781.tif
641: 343.tif
642: 512.tif
643: 1142.tif
644: 342.tif
645: 110.tif
646: 2261.tif
647: 2094.tif
648: 1443.tif
649: 130.tif
650: 1782.tif
651: 1870.tif
652: 1904.tif
653: 1821.tif
654: 1278.tif
655: 2365.tif
656: 1192.tif
657: 2017.tif
658: 1883.tif
659: 2400.tif
660: 2064.tif
661: 1787.tif
662: 494.tif
663: 1440.tif
664: 1121.tif
665: 611.tif
666: 1449.tif
667: 126.tif
668: 2250.tif
669: 903.tif
670: 2032.tif
671: 1089.tif
672: 996.tif
673: 2560.tif
674: 325.tif
675: 1446.tif
676: 2454.tif
677: 523.tif
678: 1789.tif
679: 381.tif
680: 529.tif
681: 2193.tif
682: 2538.tif
683: 1053.tif
684: 2498.

### Create test and train files
Step 2 of "Train Custom Data" [wiki](https://github.com/ultralytics/yolov3/wiki/Example:-Transfer-Learning). Create train and test `*.txt` files. Each row contains a path to an image, and remember one label must also exist in a corresponding /labels folder for each image that has targets.

In [17]:
path_to_save = '{}/yolo_data'.format(path_to_data)

# As a test, use the same random subset of data for train and test
# num_images = 40

np.random.seed(123)
train_file = open("{}/xview_train.txt".format(path_to_save), "a")
val_file = open("{}/xview_val.txt".format(path_to_save), "a")

all_imgs = os.listdir('{}/images'.format(path_to_save))
for img_filename in all_imgs:
    if (np.random.rand() < .2):
        val_file.write('../xview_data/yolo_data/images/{}\n'.format(img_filename))
    else:
        train_file.write('../xview_data/yolo_data/images/{}\n'.format(img_filename))

train_file.close()
val_file.close()